# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770354830847                   -0.53    8.0         
  2   -2.771682876135       -2.88       -1.31    1.0    128ms
  3   -2.771714178380       -4.50       -2.56    1.0    134ms
  4   -2.771714678813       -6.30       -3.63    1.0    137ms
  5   -2.771714714631       -7.45       -4.11    2.0    106ms
  6   -2.771714715222       -9.23       -5.22    1.0   94.2ms
  7   -2.771714715249      -10.57       -5.54    2.0    124ms
  8   -2.771714715250      -12.04       -6.35    1.0   98.3ms
  9   -2.771714715250      -13.16       -6.61    2.0    122ms
 10   -2.771714715250      -13.99       -7.65    1.0    118ms
 11   -2.771714715250      -15.35       -8.12    1.0    111ms


-0.00013457299183581428

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770421604796                   -0.53    9.0         
  2   -2.771771257004       -2.87       -1.31    1.0    119ms
  3   -2.771801636909       -4.52       -2.58    1.0    106ms
  4   -2.771802042561       -6.39       -3.78    1.0    135ms
  5   -2.771802073823       -7.50       -4.13    2.0    107ms
  6   -2.771802074445       -9.21       -5.25    1.0   95.0ms
  7   -2.771802074476      -10.52       -5.78    2.0    132ms
  8   -2.771802074476      -12.53       -6.01    2.0    112ms
  9   -2.771802074476      -13.25       -7.42    1.0    113ms
 10   -2.771802074476      -14.18       -7.47    2.0    129ms
 11   -2.771802074476   +  -13.92       -7.85    1.0    108ms
 12   -2.771802074476      -14.05       -9.68    1.0    111ms


0.017612221384381108

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457299183581428
Displaced dipole:  0.017612221384381108
Polarizability :   1.7746794376216921


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920642193e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551060123e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852763100448e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593849156e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787837399e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.276679646605e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.815325599600e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.815325599600e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.863382998796e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.782944573789e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 5.199299370246e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.371011931803e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.371011931803e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.